## Импортируем библиотеки и делаем предварительные настройки

In [48]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, \
    recall_score, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

Настраиваем отображение в Pandas

In [49]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

## Знакомимся с данными

In [50]:
data = pd.read_csv('./data/lab04_train.csv', index_col=[0])

In [51]:
data.head()

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_TYPE,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,APP_COMP_TYPE,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,PACK,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
333149,479990,0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.000000,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,19069.691717,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,0.981818,0.926472,0.0,324.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.139327,0.0,NaN,NaN,NaN,0.0,0.244042,NaN,0.144300,0.0,NaN,0.0,0.351852,0.166667,0.0,0.329321,0.0,NaN,45.0,NaN,45.0,0.167901,0.0,NaN,0.0,0.303750,102,NaN,NaN,5.480001,0.0,0.0,NaN,0.0,NaN,0.305106,NaN,0.345455,NaN,0.0,1.0,0.0,NaN,0.003524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303639,450480,0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.027168,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,5951.208683,NaN,NaN,NaN,0.048746,офис-менеджер,NaN,0.030303,NaN,NaN,NaN,0.0,0.0,0.030303,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,0.090909,0.292473,0.0,360.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.163876,0.0,NaN,NaN,NaN,0.0,0.465244,1.000,0.203484,0.0,1.000000,0.0,0.333333,0.333333,0.0,0.333333,0.0,NaN,45.0,NaN,45.0,0.333333,0.0,NaN,0.0,0.504160,102,NaN,NaN,6.786453,0.0,0.0,NaN,0.0,NaN,0.368799,NaN,0.454545,NaN,0.0,1.0,0.0,NaN,0.000618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
327113,473954,0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.114951,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,3774.582933,0.25,NaN,NaN,0.004519,консультант,NaN,0.363636,NaN,NaN,0.279487,0.0,0.0,0.136364,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,0.136364,0.533251,0.0,600.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.073887,0.0,NaN,NaN,NaN,0.0,0.490018,0.375,0.090256,0.0,0.317348,0.0,NaN,NaN,0.0,NaN,0.0,NaN,45.0,NaN,45.0,NaN,0.0,NaN,0.0,0.351398,102,NaN,NaN,8.528388,0.0,0.0,NaN,0.0,NaN,0.358522,NaN,0.318182,NaN,0.0,1.0,0.0,NaN,0.027484,0.0,0.0,0.0,0.0,0.0,0.0

In [52]:
data.shape

(320764, 116)

In [53]:
data.describe()

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,AMOUNT_RUB_NAS_PRC,TRANS_COUNT_SUP_PRC,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
count,320764.000000,320764.000000,286214.000000,139950.0,54850.000000,139950.0,139950.0,320763.000000,69535.000000,69535.000000,286213.000000,139950.0,100204.000000,139950.0,139950.0,320763.000000,100204.000000,139950.0,320763.000000,320763.000000,3.207630e+05,62457.000000,286213.000000,286213.000000,59823.000000,62457.000000,139950.0,139950.0,286213.000000,139950.0,320763.000000,320763.000000,59823.000000,3.207630e+05,286213.000000,286213.000000,3.207630e+05,320763.000000,104645.000000,320763.000000,104645.000000,320763.000000,320763.000000,103775.000000,7359.000000,1.135000e+03,320763.000000,7359.000000,1129.000000,7359.000000,103775.000000,320763.000000,3.207630e+05,7359.000000,1.704000e+03,320763.000000,320763.000000,179529.000000,320763.000000,320763.000000,1.795290e+05,320763.000000,230841.000000,185911.000000,320763.000000,230841.000000,63615.000000,1468.000000,86488.000000,1.408500e+05,86488.000000,185911.000000,320763.000000,140850.000000,62693.000000,320763.000000,1699.000000,2.551000e+03,320763.000000,62693.000000,320763.000000,7359.000000,320763.000000,652.000000,273904.000000,7359.000000,273904.000000,7359.000000,320763.000000,86488.000000,320763.000000,1704.000000,84414.000000,320763.000000,86488.000000,84414.000000,88692.000000,86488.000000,320763.000000,84414.000000,84414.000000,84414.000000,320763.000000
mean,368822.072125,0.105551,0.044186,0.0,50.940510,0.0,0.0,0.001312,0.416497,0.414412,0.085295,0.0,0.688782,0.0,0.0,0.000720,0.691465,0.0,0.063789,0.031481,6.616230e+04,0.444124,0.024013,0.191106,0.479673,0.437011,0.0,0.0,0.050930,0.0,0.310245,0.057133,0.472933,5.011494e+02,0.370678,0.597960,1.457711e+04,457.107921,0.701127,0.071657,0.696431,0.003865,0.001664,0.697158,22.367985,4.979336e-03,0.004901,26.272423,0.073558,25.561635,0.700298,0.212230,7.192127e+03,15.760973,1.976579e+05,0.001814,0.498683,0.642911,0.204062,0.076847,6.275881e-01,0.005753,0.628779,0.327133,0.004064,0.618276,0.028766

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320764 entries, 333149 to 222683
Columns: 116 entries, ID to TARGET
dtypes: float64(101), int64(2), object(13)
memory usage: 286.3+ MB


Наше обследование выше и `data.info()` показало нам, что у нас 13 столбцов с типом `Object`, которые нам лучше убрать

## Чистим данные

### Находим категориальные фичи

In [55]:
# Находим строковые фичи
cols_to_remove = [col for col in data.columns.values if data[col].dtype=='object']

print(cols_to_remove)

['CLNT_TRUST_RELATION', 'APP_MARITAL_STATUS', 'APP_KIND_OF_PROP_HABITATION', 'CLNT_JOB_POSITION_TYPE', 'CLNT_JOB_POSITION', 'APP_DRIVING_LICENSE', 'APP_EDUCATION', 'APP_TRAVEL_PASS', 'APP_CAR', 'APP_POSITION_TYPE', 'APP_EMP_TYPE', 'APP_COMP_TYPE', 'PACK']


Убираем категориальные фичи

In [56]:
data_cleaned = data.drop(cols_to_remove, axis=1)

Смотрим новый размер датасета

In [57]:
data_cleaned.shape

(320764, 103)

Чистим датасет от `NaN` и `Inf`

In [58]:
data_cleaned = data_cleaned.replace([np.inf, -np.inf], np.nan)

In [59]:
data_cleaned = data_cleaned.fillna(0)

Проверяем наш датафрейм после очистки

In [60]:
data_cleaned.head()

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,AMOUNT_RUB_NAS_PRC,TRANS_COUNT_SUP_PRC,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
333149,479990,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19069.691717,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.981818,0.926472,0.0,324.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.139327,0.0,0.0,0.0,0.0,0.244042,0.000,0.144300,0.0,0.000000,0.0,0.351852,0.166667,0.0,0.329321,0.0,0.0,45.0,0.0,45.0,0.167901,0.0,0.0,0.0,0.303750,0.0,0.0,5.480001,0.0,0.0,0.0,0.0,0.0,0.305106,0.0,0.345455,0.0,0.0,1.0,0.0,0.0,0.003524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303639,450480,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5951.208683,0.00,0.048746,0.030303,0.0,0.000000,0.0,0.0,0.030303,0.0,1.0,0.0,0.0,0.0,0.090909,0.292473,0.0,360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163876,0.0,0.0,0.0,0.0,0.465244,1.000,0.203484,0.0,1.000000,0.0,0.333333,0.333333,0.0,0.333333,0.0,0.0,45.0,0.0,45.0,0.333333,0.0,0.0,0.0,0.504160,0.0,0.0,6.786453,0.0,0.0,0.0,0.0,0.0,0.368799,0.0,0.454545,0.0,0.0,1.0,0.0,0.0,0.000618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
327113,473954,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3774.582933,0.25,0.004519,0.363636,0.0,0.279487,0.0,0.0,0.136364,0.0,1.0,0.0,0.0,0.0,0.136364,0.533251,0.0,600.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073887,0.0,0.0,0.0,0.0,0.490018,0.375,0.090256,0.0,0.317348,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,45.0,0.0,45.0,0.000000,0.0,0.0,0.0,0.351398,0.0,0.0,8.528388,0.0,0.0,0.0,0.0,0.0,0.358522,0.0,0.318182,0.0,0.0,1.0,0.0,0.0,0.027484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
384197,531038,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,148568.263850,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.666667,0.761035,0.0,732.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.165847,0.0,0.0,0.0,0.0,0.497541,0.000,0.000000,0.0,0.000000,0.0,0.000000

In [61]:
data_cleaned.describe()

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,AMOUNT_RUB_NAS_PRC,TRANS_COUNT_SUP_PRC,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
count,320764.000000,320764.000000,320764.000000,320764.0,320764.000000,320764.0,320764.0,320764.000000,320764.000000,320764.000000,320764.000000,320764.0,320764.000000,320764.0,320764.0,320764.000000,320764.000000,320764.0,320764.000000,320764.000000,3.207640e+05,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.0,320764.0,320764.000000,320764.0,320764.000000,320764.000000,320764.000000,3.207640e+05,320764.000000,320764.000000,3.207640e+05,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,3.207640e+05,320764.000000,3.207640e+05,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000,320764.000000
mean,368822.072125,0.105551,0.039427,0.0,8.710725,0.0,0.0,0.001312,0.090288,0.089836,0.076107,0.0,0.215170,0.0,0.0,0.000720,0.216008,0.0,0.063789,0.031481,6.616209e+04,0.086477,0.021427,0.170521,0.089460,0.085092,0.0,0.0,0.045444,0.0,0.310244,0.057132,0.088203,5.011479e+02,0.330751,0.533551,1.457707e+04,457.106496,0.228733,0.071657,0.227202,0.003865,0.001664,0.225548,0.513169,0.000018,0.004901,0.602745,0.000259,0.586438,0.226564,0.212229,7.192105e+03,0.361590,1.050021e+03,0.001814,0.498681,0.359832,0.204061,0.076847,0.351256,0.005753,0.45250

In [62]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320764 entries, 333149 to 222683
Columns: 103 entries, ID to TARGET
dtypes: float64(101), int64(2)
memory usage: 254.5 MB


Достаточный EDA. Считаю, что за некоторые категориальные данные можно было побороться.
Вопрос времени и сил. Задача выполнена +

## Строим модель RandomFortest

Отделяем таргет от датафрема

In [63]:
y = data_cleaned['TARGET']
data_cleaned_no_target = data_cleaned.drop('TARGET', axis=1)

Определяем выборки для тренировки модели

In [64]:
X_train, X_test, y_train, y_test = train_test_split(
    data_cleaned_no_target,
    y,
    test_size=0.5,
    random_state=20
)

In [75]:
model = RandomForestClassifier(n_estimators=100, max_depth=15, n_jobs=4, random_state=20)
model.fit(X_train, y_train)

pred = model.predict_proba(X_test)[:,1]

Проверяем результаты по метрике ROC_AUC

In [67]:
roc_auc_score(y_test, pred)

0.8279576384014686

### Прогоняем на тестовых данных

In [68]:
test_data = pd.read_csv('./data/lab04_test.csv', index_col=[0])

Чистим данные аналогично основному датафрейму

In [69]:
cols_to_remove = [col for col in test_data.columns.values if test_data[col].dtype=='object']

test_data_cleaned = test_data.drop(cols_to_remove, axis=1)
test_data_cleaned = test_data_cleaned.replace([np.inf, -np.inf], np.nan)
test_data_cleaned = test_data_cleaned.fillna(0)

In [70]:
y_test_result = model.predict_proba(test_data_cleaned)[:,1]

## Строим модель XGBoost

In [71]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=20)
xgb_model.fit(X_train, y_train)

pred = xgb_model.predict_proba(X_test)[:,1]

Проверяем результаты по метрике ROC_AUC

In [72]:
roc_auc_score(y_test, pred)

0.8440109534645336

In [73]:
y_test_result = xgb_model.predict_proba(test_data_cleaned)[:,1]

## Выгружаем результаты

In [74]:
submission = pd.DataFrame()
submission['id'] = test_data_cleaned['ID']
submission['target'] = y_test_result
submission['target'] = submission['target'].astype(float)
submission.to_csv(f'./results/lab04s.csv', sep='\t', index=False)

Хороший, последовательный, структурированный код. 5+